### Library Import

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt

### Setting U-Net 

In [ ]:
class UNet(nn.Module):
    def __init__(self, in_channels=3, out_features=64, num_classes=2):
        super(UNet, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, out_features, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(out_features, out_features * 2, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(out_features * 2, out_features, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(out_features, num_classes, kernel_size=2, stride=2)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

### Set DataClass

In [ ]:
class MedicalDataset(Dataset):
    def __init__(self, image_dir, labels, transform=None):
        self.image_dir = image_dir
        self.labels = labels
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, self.images[index])
        image = Image.open(img_path).convert("RGB")
        label = self.labels[self.images[index]]

        if self.transform:
            image = self.transform(image)

        return image, label

### DataLoader

In [ ]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

labels = {'image1.jpg': 0, 'image2.jpg': 1}  # 예시 레이블
train_dataset = MedicalDataset(image_dir='/path/to/train/images', labels=labels, transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=4, shuffle=True, num_workers=2)

### Setting loss function ald optimization

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet(in_channels=3, out_features=64, num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

### Training Loop

In [ ]:
num_epochs = 35
for epoch in range(num_epochs):
    model.train()
    for data, targets in train_loader:
        data = data.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

### Save Model

In [ ]:
torch.save(model.state_dict(), 'UNet_classification_model.pth')